In [ ]:
#running on tensorflow backend
#tensorflow version: tensorflow-gpu 1.13.1
#keras version: 2.2.4
#numpy version: 1.16.3
#used pretrained model: MobileNetV2
'''Sandler, M., Howard, A., Zhu, M., Zhmoginov, A., & Chen, L. (2018). 
    MobileNetV2: Inverted Residuals and Linear Bottlenecks. 
    Arxiv E-Prints, arXiv:1801.04381.'''
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
tf.__version__

In [ ]:
# Dataset paths
train_path = 'vegetables'
valid_path = 'vegetables_val'
test_path = 'vegetables_test'


IMAGE_SIZE = 224      # Image height and width
BATCH_SIZE = 64       # How many images will be processed each time

# Generate training set, validation set and test set
train_generator = ImageDataGenerator(
    #horizontal_flip=True,
    #rotation_range=30,
    preprocessing_function=keras.applications.mobilenet_v2.preprocess_input).flow_from_directory(
    train_path, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE)
val_generator = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet_v2.preprocess_input).flow_from_directory(
    valid_path, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE)
test_batches = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet_v2.preprocess_input).flow_from_directory(
    valid_path, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE, shuffle=False)

for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

In [ ]:
# Export the label file
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

In [ ]:
# Transfer learning step, freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False
for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
# Total class numbers
CLASS_NUM = 84
# Create the squential model
model = models.Sequential()
 
# Add the MobileNetV2 convolutional base model
model.add(base_model)
 
# Add new layers
model.add(layers.AveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(CLASS_NUM, activation='softmax'))
 
# Show a summary of the model.
model.summary()

In [ ]:
#compile the model using Adam optimizer
model.compile(optimizer=optimizers.Adam(0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

In [ ]:
#train the model
epochs = 5

history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)


In [ ]:
#draw a graph for accuracy and loss
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,3.5])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# Fine tuning step 1: unfreeze layers in base model
for layer in base_model.layers:
    layer.trainable = True

# Fine tuning top 55 layers
fine_tuning = 100

# Fine tuning Step 2: Freeze the bottom 100 layers
for layer in base_model.layers[:fine_tuning]:
  layer.trainable =  False

for i, layer in enumerate(base_model.layers):
   print(i, layer.name, layer.trainable)

In [ ]:
#compile the model
model.compile(loss='categorical_crossentropy',
              optimizer = optimizers.Adam(0.0001),
              metrics=['accuracy'])
filepath="mobilenetV2_model_best.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# print the number of trainable variables
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

In [ ]:
# train the model
history_fine = model.fit(train_generator, 
                         epochs=20,
                         validation_data=val_generator,
                         callbacks=[checkpoint])

In [ ]:
# draw a graph for accuracy and loss
acc = history_fine.history['acc']
val_acc = history_fine.history['val_acc']

loss = history_fine.history['loss']
val_loss = history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0, 1.5])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# save the model
keras_file = "food_mobilenetV2_model.h5"
tf.keras.models.save_model(model, keras_file)

In [ ]:
# make a single prediction
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

img = load_img('vegetables_test/carrot_cake/1381243.jpg',target_size=(224, 224))  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 224, 224)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 224, 224)
x = keras.applications.mobilenet.preprocess_input(x)
prediction = model.predict(x)
maximum = max(prediction[0])
[i for i, j in enumerate(prediction[0]) if j == maximum]

In [ ]:
#the fuction to draw a confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
#make prediction on test images
test_labels = test_batches.classes
predictions = model.predict_generator(test_batches, steps = 100,verbose=0)

In [ ]:
#print the labels
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))
test_batches.class_indices

In [ ]:
#plot the confusion matrix
cm_plot_labels = [i for i in range(84)]
plt.figure()
fig = plt.gcf()
fig.set_size_inches(28, 28)
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')